# Loading required packages


In [1]:
import numpy as np
from pylab import *
from astropy.io import ascii
from astropy import units
import matplotlib
import matplotlib.pyplot as plt
import emcee

# Calculating flux @ 5 keV

In [2]:
t, tpos, tneg ,  flux ,fluxpos , fluxneg = np.loadtxt( 'xrt_0.3-10keV.txt' ,dtype=float,unpack=True,usecols=[0,1,2,3,4,5])
gamma, gammapos,gammaneg= np.loadtxt( 'gamma_0.3-10keV.txt' ,dtype=float,unpack=True,skiprows=0,usecols=[3,4,5])

E= 5 #keV
num=4.3*10**11*(2-gamma)*(np.power(E,1-gamma))
den=((10**(2-gamma))-(0.3**(2-gamma)))
flux=flux*(num/den)*1e-3
fluxpos=fluxpos*(num/den)*1e-3
fluxneg=fluxneg*(num/den)*1e-3
ferr=(fluxpos+(-fluxneg))/2
f=open('fluxdensity@5keV_xrt_190829A.txt','w')
f.write('#time(s)\t tpos(s)\t tneg(s)\t  E(keV)\t  F.D.(mJy)\t fluxpos(mJy)\t fluxneg(mJy)\n')
for i in range(len(t)):
       f.write('%1.8e\t %1.8e\t% 1.8e \t%1.8e \t%1.8e \t%1.8e \t%1.8e \n' %(t[i], tpos[i], tneg[i] ,  E,flux[i] ,fluxpos[i] , fluxneg[i]))
f.close()


# Optical : Converting mag to flux (galactic corrected)

In [3]:
t0, m0, merr0, w0, zp0 = np.loadtxt( 'GRB190829A_op.txt' ,dtype=float,unpack=True,skiprows=0,usecols=[0,1,2,3,4])
t0= t0*3600
Extn=0.0420
 # galactic extinction
A_V=3.1*Extn
# finding the value of extinction in given band (ref :Cardelli et al.)
#p=eff_wavelength
size=len(w0)
p=1/w0
i=0
Rv=3.1
p1=np.zeros(size)
p2=np.zeros(size)
r=np.zeros(size)
a=np.zeros(size)
b=np.zeros(size)
A=np.zeros(size)
f0=np.zeros(size)
ferr0=np.zeros(size)
for i in range (0, size) :
    if (0.30<p[i] <1.1):
        r[i]=(p[i]-1.82)
        a[i]=1+0.17699*r[i]-0.50447*(r[i]**2)-0.02427*(r[i]**3)+0.72085*(r[i]**4)+0.01979*(r[i]**5)-0.77530*(r[i]**6)+0.32999*(r[i]**7)
        b[i]=1.41338*r[i]+2.28305*(r[i]**2)+1.07233*(r[i]**3)-5.38434*(r[i]**4)-0.62251*(r[i]**5)+5.30260*(r[i]**6)-2.09002*(r[i]**7)
        A[i]=A_V*(a[i]+b[i]/Rv)
        m0[i]= m0[i] - A[i]
        f0[i]=((10**(-m0[i]/2.5))*zp0[i])
        ferr0[i]=((zp0[i])*(0.4*2.302585093*merr0[i])*(10**(-0.4*m0[i])))
    if (1.1<p[i]<3.3):
        a[i]=0.574*p[i]**(1.61)
        b[i]=-0.527*p[i]**(1.61)
        A[i]=A_V*(a[i]+b[i]/Rv)
        m0[i]= m0[i] - A[i]
        f0[i]=((10**(-m0[i]/2.5))*zp0[i])
        ferr0[i]=((zp0[i])*(0.4*2.302585093*merr0[i])*(10**(-0.4*m0[i])))
    if (3.3<p[i]<5.9):    
        a[i] = 1.752 - 0.316*p[i] - 0.104/((p[i] - 4.67)*2+ 0.341) 
        b[i] = -3.090 + 1.825*p[i] + 1.206/((p[i] - 4.62)*2+ 0.263) 
        A[i]=A_V*(a[i]+b[i]/Rv) 
        m0[i]= m0[i] - A[i]
        f0[i]=((10**(-m0[i]/2.5))*zp0[i])
        ferr0[i]=((zp0[i])*(0.4*2.302585093*merr0[i])*(10**(-0.4*m0[i])))
    

#correcting for galactic extinction
#m0=m0-A
print(w0,A)
f0=f0*10**(3)
ferr0=ferr0*10**(3)
f=open('fluxdensity_op_190829A.txt','w')
f.write('#time(sec)\t   wavelength(micron)\t  zeropt(Jy)\t  mag\t   magerr\t   F.D.(Jy)\t fluxEer(Jy)\n')

for i in range(len(t0)):
       f.write('%1.8e\t %1.8e\t%1.8e \t%1.8e \t%1.8e \t%1.8e \t%1.8e \n' %(t0[i], w0[i],zp0[i],m0[i], merr0[i],f0[i],ferr0[i]))
f.close()



[0.641 0.798 1.22  1.22  1.22  1.63  1.63  1.63  2.19  2.19  2.19  0.483
 0.483 0.483 0.483 0.483 0.483 0.626 0.626 0.626 0.626 0.626 0.626 0.626
 0.767 0.767 0.767 0.767 0.767 0.767 0.767 0.91  0.91  0.91  0.91  0.91 ] [0.10763436 0.07564282 0.09414394 0.09414394 0.09414394 0.32349344
 0.32349344 0.32349344 0.79626711 0.79626711 0.79626711 0.16976082
 0.16976082 0.16976082 0.16976082 0.16976082 0.16976082 0.11181696
 0.11181696 0.11181696 0.11181696 0.11181696 0.11181696 0.11181696
 0.08062539 0.08062539 0.08062539 0.08062539 0.08062539 0.08062539
 0.08062539 0.06127131 0.06127131 0.06127131 0.06127131 0.06127131]


# Creating file for modeling

In [4]:
c = 2.4180e+17
freq = E*c
c0 = 2.9979e+14 
freq0 = c0/w0
print(freq0)
f=open('grbdata_190829A.txt','w')
f.write('#tobs(s)\t  nuobs(Hz)\t   f(mJy)\t  ferr(mJy)\n #Optical \n')
for i in range(len(t0)):
       f.write('%1.8e\t %1.8e\t% 1.8e\t% 1.8e  \n' %(t0[i], freq0[i],f0[i],ferr0[i]))
f.write('# X-ray \n')
for i in range(len(t)):
       f.write('%1.8e\t %1.8e\t% 1.8e\t% 1.8e  \n' %(t[i], freq,flux[i],ferr[i]))
f.close()

[4.67691108e+14 3.75676692e+14 2.45729508e+14 2.45729508e+14
 2.45729508e+14 1.83920245e+14 1.83920245e+14 1.83920245e+14
 1.36890411e+14 1.36890411e+14 1.36890411e+14 6.20683230e+14
 6.20683230e+14 6.20683230e+14 6.20683230e+14 6.20683230e+14
 6.20683230e+14 4.78897764e+14 4.78897764e+14 4.78897764e+14
 4.78897764e+14 4.78897764e+14 4.78897764e+14 4.78897764e+14
 3.90860495e+14 3.90860495e+14 3.90860495e+14 3.90860495e+14
 3.90860495e+14 3.90860495e+14 3.90860495e+14 3.29439560e+14
 3.29439560e+14 3.29439560e+14 3.29439560e+14 3.29439560e+14]


# Dealing with upper limits

In [6]:
t0, m0, merr0, w0, zp0 = np.loadtxt( 'GRB051221A_op_uls.txt' ,dtype=float,unpack=True,skiprows=0,usecols=[0,1,2,3,4])
#Extn=0.0650
 # galactic extinction
A_V=3.1*Extn
# finding the value of extinction in given band (ref :Cardelli et al.)
#p=eff_wavelength
size=len(w0)
p=1/w0
i=0
Rv=3.1
p1=np.zeros(size)
p2=np.zeros(size)
r=np.zeros(size)
a=np.zeros(size)
b=np.zeros(size)
A=np.zeros(size)
f0=np.zeros(size)
ferr0=np.zeros(size)
for i in range (0, size) :
    if (0.30<p[i] <1.1):
        r[i]=(p[i]-1.82)
        a[i]=1+0.17699*r[i]-0.50447*(r[i]**2)-0.02427*(r[i]**3)+0.72085*(r[i]**4)+0.01979*(r[i]**5)-0.77530*(r[i]**6)+0.32999*(r[i]**7)
        b[i]=1.41338*r[i]+2.28305*(r[i]**2)+1.07233*(r[i]**3)-5.38434*(r[i]**4)-0.62251*(r[i]**5)+5.30260*(r[i]**6)-2.09002*(r[i]**7)
        A[i]=A_V*(a[i]+b[i]/Rv)
        m0[i]= m0[i] - A[i]
        f0[i]=((10**(-m0[i]/2.5))*zp0[i])
        ferr0[i]=((zp0[i])*(0.4*2.302585093*merr0[i])*(10**(-0.4*m0[i])))
    if (1.1<p[i]<3.3):
        a[i]=0.574*p[i]**(1.61)
        b[i]=-0.527*p[i]**(1.61)
        A[i]=A_V*(a[i]+b[i]/Rv)
        m0[i]= m0[i] - A[i]
        f0[i]=((10**(-m0[i]/2.5))*zp0[i])
        ferr0[i]=((zp0[i])*(0.4*2.302585093*merr0[i])*(10**(-0.4*m0[i])))
    if (3.3<p[i]<5.9):    
        a[i] = 1.752 - 0.316*p[i] - 0.104/((p[i] - 4.67)*2+ 0.341) 
        b[i] = -3.090 + 1.825*p[i] + 1.206/((p[i] - 4.62)*2+ 0.263) 
        A[i]=A_V*(a[i]+b[i]/Rv) 
        m0[i]= m0[i] - A[i]
        f0[i]=((10**(-m0[i]/2.5))*zp0[i])
        ferr0[i]=((zp0[i])*(0.4*2.302585093*merr0[i])*(10**(-0.4*m0[i])))
   

#correcting for galactic extinction
#m0=m0-A
print(w0,A)
f0=f0*10**(3)
ferr0=ferr0*10**(3)
f=open('fluxdensity_op_ul_061201.txt','w')
f.write('#time(sec)\t   wavelength(micron)\t  zeropt(Jy)\t  mag_ul\t   magerr\t   F.D._ul(mJy)\t fluxEer_ul(mJy)\n')

for i in range(len(t0)):
       f.write('%1.8e\t %1.8e\t%1.8e \t%1.8e \t%1.8e \t%1.8e \t%1.8e \n' %(t0[i], w0[i],zp0[i],m0[i], merr0[i],f0[i],ferr0[i]))
f.close()



[0.4346 0.2684 0.2086 0.2246 0.3386 0.5411 0.641  0.626  0.626  0.767
 0.767  0.91   0.91  ] [ 0.32913597  0.33722955  0.537303   -0.49190507  0.49193056  0.23127195
  0.17605906  0.1829006   0.1829006   0.1318801   0.1318801   0.10022235
  0.10022235]


In [8]:
c = 2.4180e+17
freq = E*c
c0 = 2.9979e+14 
freq0 = c0/w0
print(freq0)
f=open('grbdata_051221A.txt','a')
f.write('#tobs(s)\t  nuobs(Hz)\t   f(mJy)\t  ferr(mJy)\n#Optical upper limits \n')
for i in range(len(t0)):
       f.write('%1.8e\t %1.8e\t% 1.8e\t% 1.8e \n' %(t0[i], freq0[i],0.000,f0[i]))        
f.close()

[6.89806719e+14 1.11695231e+15 1.43715244e+15 1.33477293e+15
 8.85380981e+14 5.54038071e+14 4.67691108e+14 4.78897764e+14
 4.78897764e+14 3.90860495e+14 3.90860495e+14 3.29439560e+14
 3.29439560e+14]


In [6]:
t,nu,f_r,ferr_r = np.loadtxt( 'radio.txt' ,dtype=float,unpack=True,skiprows=0,usecols=[0,1,2,3])
t=t*3600*24
f=open('grbdata_190829A.txt','a')
f.write('#tobs(s)\t  nuobs(Hz)\t   f(mJy)\t  ferr(mJy)\n#Radio\n')
for i in range(len(t)):
       f.write('%1.8e\t %1.8e\t% 1.8e\t% 1.8e \n' %(t[i], nu[i],f_r[i],ferr_r[i]))        
f.close()